In [2]:
import pandas as pd
import numpy as np
import os

comp1 = pd.read_excel("input_computational-1.xlsx")
comp2 = pd.read_excel("input_computational-2.xlsx")

comp = pd.concat([comp1, comp2], ignore_index=True)


def get_extrapolated_energy(group):
    from scipy.stats import linregress

    if len(group) < 2:
        return pd.Series(
            {
                "delta_h": group.iloc[0]["delta_h"],
                "delta_h_std": None,
                "method_calc": group.iloc[0]["method_calc"].lower(),
                "extrapolation_file": None,
                "repeating_units": group.iloc[0]["repeating_units"],
            }
            | {
                k: group.iloc[0][k]
                for k in group.columns
                if k not in group_cols + ["method_calc", "delta_h", "repeating_units"]
            }
        )

    mask = group["repeating_units"] != 100
    x = 1 / group["repeating_units"][mask]
    y = group["delta_h"][mask]
    slope, intercept, r_value, p_value, std_err = linregress(x, y)

    # create a dataframe with the repeating units and delta_h
    df_fit = pd.DataFrame(
        {
            "monomer_smiles": group["monomer_smiles"].iloc[0],
            "polymerisation_type": group["polymerisation_type"].iloc[0],
            "doi": group["doi"].iloc[0],
            "medium": group["medium"].iloc[0],
            "solvent": group["solvent"].iloc[0],
            "initiator_smiles": group["initiator_smiles"].iloc[0],
            "polymer_smiles": group["polymer_smiles"].iloc[0],
            "temperature": group["temperature"].iloc[0],
            "repeating_units": group["repeating_units"][mask],
            "delta_h": group["delta_h"][mask],
        }
    )

    # write the fit df based on the DOI + monomer name. Check if a file exists already and if so number it 1, 2, 3 etc
    # also make sure the filename doesn't contain any bad symbols
    filename = f"{group.iloc[0]['doi']}_{group.iloc[0]['monomer_smiles']}_1.csv"
    for sym in ["/", ":", "(", ")", "[", "]", "{", "}", " "]:
        filename = filename.replace(sym, "_")

    if os.path.exists(filename):
        i = 2
        while os.path.exists(f"{filename[:-4]}_{i}.csv"):
            i += 1
        filename = f"{filename[:-4]}_{i}.csv"

    df_fit.to_csv("extrapolation/" + filename, index=False)

    # if group.iloc[0]["monomer_smiles"] == "S=C1SCCSC1C2=CC=CC=C2":
    # fig, ax = plt.subplots()
    # ax.plot(x, y, 'o')
    # # plot the linear fit line
    # ax.plot(np.linspace(-0.1, 0.6, 10), slope * np.linspace(-0.1, 0.6, 10) + intercept, 'r-', label=f'y={slope:.2f}x+{intercept:.2f}')
    # plt.show()
    # print(np.sqrt(std_err))

    return pd.Series(
        {
            "delta_h": intercept,
            "delta_h_std": np.sqrt(std_err),
            "method_calc": "ie_extrap",
            "extrapolation_file": filename,
            "repeating_units": 100,
        }
        | {
            k: group.iloc[0][k]
            for k in group.columns
            if k not in group_cols + ["method_calc", "delta_h", "repeating_units"]
        }
    )


group_cols = [
    "monomer_smiles",
    "polymerisation_type",
    "doi",
    "medium",
    "solvent",
    "initiator_smiles",
    "polymer_smiles",
    "temperature",
]
merged = (
    comp.groupby(group_cols, dropna=False).apply(get_extrapolated_energy).reset_index()
)

columns = [
    "monomer_smiles",
    "polymerisation_type",
    "is_experimental",
    "polymer_smiles",
    "topology",
    "repeating_units",
    "initiator_smiles",
    "medium",
    "solvent",
    "solvent_model",
    "monomer_state",
    "polymer_state",
    "temperature",
    "pressure",
    "method",
    "method_calc",
    "delta_h",
    "delta_h_std",
    "delta_s",
    "delta_g",
    "ceiling_temperature",
    "doi",
    "extrapolation_file",
    "date",
    "url",
    "functional",
    "basis_set",
    "dispersion",
    "forcefield",
    "flag",
    "comment",
    "initial_monomer_conc",
    "bulk_monomer_conc",
    "dispersity",
    "degree_of_polymerisation",
    "number_average_molar_mass",
    "mass_average_molar_mass",
]

merged[columns].sort_values(by=["doi", "monomer_smiles"]).to_excel(
    "../input_computational.xlsx", index=False
)

/var/folders/0b/8_7qpw094rgbnbv0snpldtr00000gn/T/ipykernel_25763/3026429286.py:100: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  comp.groupby(group_cols, dropna=False).apply(get_extrapolated_energy).reset_index()
